## Core

In [ ]:
#| default_exp core

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from dotenv import load_dotenv
import os

In [ ]:
#| hide
load_dotenv();

In [ ]:
#| export
import json
from typing import List, Dict, Tuple
import textwrap
import claudette
from IPython import get_ipython
from IPython.display import display, clear_output, Markdown, Javascript
from friendlly.utils import ExecCBs

In [ ]:
#| export

# A single cell can contain multiple messages.
# A message is either a user message (starts with %fr) or a bot message (starts with #).
# Both can be multiline.
def parse_cell(
    cell: str # The raw body of the cell
) -> Tuple[List[Dict[str, str]], int]:
    """
    A single cell can contain multiple messages.
    A message is either a user message (starts with %fr) or a bot message (starts with #).
    Both can be multiline.

    Returns: a list of messages (with 'role' and 'content') and the number of %fr magics in the cell
    """
    parsed_lines = []
    num_magic = 0
    for line in cell.split('\n'):
        if line.startswith('%fr'):
            message = {'role': 'user', 'content': line[3:].strip()}
            num_magic += 1
        elif line.strip().startswith('#'):
            message = {'role': 'assistant', 'content': line[1:].strip()}
        else: continue

        if not parsed_lines or parsed_lines[-1]['role'] != message['role']:
            parsed_lines.append(message)
        else:
            parsed_lines[-1]['content'] += ("\n" + message['content'])

    return parsed_lines, num_magic

In [ ]:
#| exporti
#| hide

models = [
    'claude-3-opus-20240229',
    'claude-3-5-sonnet-20240620',
    'claude-3-haiku-20240307',
]
chat_client = claudette.Client(model=models[1])

magic_count = 0
messages = []

In [ ]:
#| export
def fr_line(line: str):
    """The magic function for the %fr magic command."""
    global magic_count, messages
    ip = get_ipython()
    # raw_cell = ip.history_manager.input_hist_raw[-1]
    raw_cell = ip.get_parent()["content"]["code"]

    # The cell might have multiple %lm magics, but we only want to process the last one.
    # Presumably, the previous ones would have been processed already.
    if magic_count <= 0:
        messages, magic_count = parse_cell(raw_cell)

    # This is the last %lm magic invocation of the cell.
    # But we ignore cells that don't have a user message as the last message.
    if magic_count == 1 and len(messages) > 0 and messages[-1]['role'] == 'user' and messages[-1]['content'].strip():
        reply = ""
        display_id = display(Markdown("🚀..."), display_id=True)
        try:
            r = chat_client([m['content'] for m in messages], stream=True)
            for token in r:
                reply += token
                display_id.update(Markdown(reply))

            if reply:
                reply = textwrap.fill(text=reply, width=100, initial_indent="# ", subsequent_indent="# ")
                raw_cell += f"\n{reply}\n\n%fr "
                ip.set_next_input(raw_cell, replace=True)

            clear_output()

        except BaseException as e:
            display_id.update(Markdown(f"🚫 {repr(e)}"))


    magic_count -= 1

## Friend**LL**y

In [ ]:
#| eval: False
ip = get_ipython()
ip.register_magic_function(fr_line, 'line', magic_name='fr')

In [ ]:
%fr Hello there! My name is Alex.

# Hello Alex! It's nice to meet you. How can I assist you today? Is there anything specific you'd
# like to talk about or any questions you have?

%fr

In [ ]:
from friendlly.utils.nbclassic import patch_kernel, add_cell, update_cell, render_cell, execute_cell, get_index

In [ ]:
#| export

sp="""
You are ConGusto, a very knowledgeable and intelligent research assistant.
Fulfill simple requests right away.
Use <thought>, it's not shown to the user. You can use <thought> multiple times.

When appropriate, use Jupyter notebook. It has python 3.10+ and many packages installed.
Use <code> to add a new code cell.
Use <run> to execute the added cell. You will receive all its output on the next turn.

Avoid putting too much in one cell. Prefer short cells that can be tested quickly.
This way you will be able to check that the cell works as intended and continue. 

You can use the usual display(), print(), or just place the value on the last line of the cell.
Make sure to not add anything after that value, as it won't be retuerned if it's not on the last line.
Only output a result once with one method.
Avid using .show() and close matplot figures before the end of the cell - use an explicit display().

After code execution, if the request has been fullfilled, reply only with <done>, do not explain the obvious.
The user will ask follow-up questions if needed. They want you to do only what they asked for.
You should also end your text with <done> if there is no need to run code.

To show an example in python without running the code, use
```python
python code here
```

For example:
User: Calculate square root of pi
Assistant: <code>
import math
math.sqrt(math.pi)
</code><run>
User: <stdio>1.7724538509055159<stdio>
Assistand: <done>

User: Display cat.jpg
Assistant: <thought>I can use the PIL library for this.</thought><code>
from PIL import Image
Image.open("cat.jpg")
</code><run>
User: <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=128x127>
User: <image message>
Assistant: <done>

When writing code, use advanced python features.
Assume all packages are instlled.
Don't forget to import modules before using them.
If you realize that you need to use a module that is not imported, import in in place.
"""


In [ ]:
# from PIL import Image
# from pathlib import Path
# fn = Path("cat.jpg")
import cairosvg
import xml.etree.ElementTree as ET
from io import BytesIO
import base64

def svg_to_png(svg_content, max_size):
    root = ET.fromstring(svg_content)

    
    def parse_dimension(value):
        if value.endswith('%'):
            return float(value[:-1]) / 100 * max_size
        return float(re.findall(r'[\d.]+', value)[0])

    width = parse_dimension(root.attrib.get('width', str(max_size)))
    height = parse_dimension(root.attrib.get('height', str(max_size)))
    
    viewBox = root.attrib.get('viewBox')
    if viewBox:
        _, _, vb_width, vb_height = map(float, viewBox.split())
        if width == max_size:
            width = vb_width
        if height == max_size:
            height = vb_height

    scale = min(max_size / width, max_size / height)
    new_width = int(width * scale)
    new_height = int(height * scale)

    # Render PNG
    png =  cairosvg.svg2png(bytestring=svg_content, output_width=new_width, output_height=new_height)
    return base64.b64encode(png)

def display_to_messages(dsp):
    plain_text = dsp.get("text/plain")
    text = list(filter(lambda x:x[1], [
        ("text-html", dsp.get("text/html")),
        ("latex", dsp.get("text/latex")),
        ("markdown", dsp.get("text/markdown")),
        ("json", dsp.get("application/json")),
        ("js", dsp.get("application/javascript"))
    ]))
    
    images = list(filter(lambda x:x, [ dsp.get("image/png"), dsp.get("image/jpeg") ]))
    svg = dsp.get("image/svg+xml")
    if svg: images.append(svg_to_png(svg, 256))
  
    messages = [f"<{t[0]}>{t[1]}</{t[0]}>" for t in text]
    
    if not len(messages):
        messages = [plain_text]
        
    claudette_msgs = [claudette.text_msg(m) for m in messages]
    if images:
        claudette_msgs.append(claudette.img_msg(base64.b64decode(images[0]))) # Grab only one image

    return claudette_msgs


In [ ]:
#| export

class CellChat():
    def __init__(self, model="claude-3-5-sonnet-20240620", sp=None):
        if sp is not None: self.sp = sp
        self.history = []
        self.client = claudette.Client(model=model)


    def update_markdown(self, text):
        if self.md_idx is None:
            self.md_idx = self.idx+1
            add_cell(self.md_idx, "markdown")
        update_cell(self.md_idx, text)
        render_cell(self.md_idx)

    def update_code(self, code):
        if self.code_idx is None:
            self.code_idx = self.md_idx+1 if (self.md_idx is not None) else self.idx+1
            add_cell(self.code_idx, "code")
        update_cell(self.code_idx, code)

    def go(self, idx, message, autorun=True):
        assert idx >= 0
        self.idx = idx
        self.md_idx = self.code_idx = None
        self.history.append(message)
        
        tokens = []
        try:
            md_reply = ""
            code_reply = None
            add_md_cell = True
            r = self.client(claudette.mk_msgs(self.history),
                            sp=self.sp,
                            stream=True,
                            stop=["<done>", "<run>"])
            for token in r:
                tokens.append(token)
                
                md_reply += token
                if "<code>" in md_reply:
                    md_reply, code_reply = md_reply.split("<code>")

                if md_reply != "":
                    self.update_markdown(md_reply.strip())
                if code_reply is not None:
                    break

            if code_reply is not None:
                for token in r:
                    tokens.append(token)
                    code_reply += token
                    self.update_code(code_reply.strip())
                
            if self.client.stop_reason == "stop_sequence":
                tokens += self.client.stop_sequence
            self.history.append(claudette.mk_msg("".join(tokens), role="assistant"))
            
            if autorun and self.code_idx is not None:
                # This schedules the cell for execution once this one is finished.
                execute_cell(self.code_idx)
                return True
            
        except BaseException as e:
            self.display_handle.update(Markdown(f"🚫 {repr(e)}"))
            get_ipython().showtraceback()



In [ ]:
chat = None
cbs = None


import cairosvg
from io import BytesIO

def followup(stdout, stderr, dsp, result, tracebacks):
    text_message = ""
    
    assert len(tracebacks) <= 1
    if result.error_before_exec:
        text_message += "<error_before_exec>" + tracebacks[0]["stb"] + "</error_before_exec>"
    
    if result.error_in_exec:
        text_message += "<error_in_exec>" + tracebacks[0]["stb"] + "</error_in_exec>"
    
    if stdout: text_message += f"<stdout>{stdout}</stdout>"
    if stderr: text_message += f"<stderr>{stderr}</stderr>"
        
    if dsp is None: dsp = []
    display_messages = []
    for d in dsp:
        msgs = display_to_messages(d)
        for msg in msgs:
            display_messages.append(msg)

        
    result_message = []
    if result.result:
        formatted, _ = get_ipython().display_formatter.format(result.result)
        for key in ("image/jpg", "image/png"):
            if key in formatted: formatted[key] = base64.b64encode(formatted[key]).decode("ascii")

        result_message = display_to_messages(formatted)
        
    combined_messages = []
    if text_message: combined_messages += [claudette.text_msg(text_message)]
    if display_messages: combined_messages += display_messages
    if result_message: combined_messages += result_message
#     raise ValueError
    message = claudette.mk_msg(combined_messages)
    
#     raise "hello"
    to_exec = chat.go(get_index(), message)
    if to_exec:
            cbs = ExecCBs(followup)
            cbs.register()

def fr_cell(line=None, cell=None):
    global cbs
    global chat
    chat = CellChat(sp=sp)

    to_exec = chat.go(get_index(), claudette.mk_msg(cell, role="user"))
    if to_exec:
        cbs = ExecCBs(followup)
        cbs.register()

get_ipython().register_magic_function(fr_cell, 'cell', magic_name='fr')

In [ ]:
#| export

# def load_ipython_extension(ipython):
#     patch_kernel()
#     ipython.register_magic_function(fr_line, 'line', magic_name='fr')
#     ipython.register_magic_function(fr_cell, 'cell', magic_name='fr')

# def unload_ipython_extension(ipython):
#     if cbs: cbs.unregister()

In [ ]:
#| eval: false
# load_ipython_extension(get_ipython())

In [ ]:
%%fr
Download an image of a cat and show it, describe what you see.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("hello")

hello


sdfsf